In [ ]:
# Imports
import numpy as np
import pandas as pd
from db import DB
from db import list_profiles
from datetime import *
import os
import glob
import time
from os import listdir
from os.path import isfile, join
from openpyxl import Workbook
from openpyxl import load_workbook
import xlsxwriter
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# User config
user_config = {
    'base_path': r'C:\Users\brandon.terrebonne\Desktop\outbound_file_screening'
}

## Defining fucntions ##
# Sorts out instances where the buyer sends eslap files containing non-compliant field names (e.g. 'INVOICEID' vs 'invoice_id')
def buyer_outbound_mappings(buyer_configs_data, buyer_name, table_field_name, eslap_col_names):
    buyer_data = buyer_configs_data[buyer_configs_data['upload_directory'] == buyer_name]
    index_list = buyer_data.index.tolist()
    buyer_index = index_list[0]
    pre_transform_col_names = []
    pre_transform_col_names_values = []
    for x in eslap_col_names:
        try:
            trans_name = buyer_data.loc[buyer_index][table_field_name]['TransformItems'][x]['IncomingColumnName']
            pre_transform_col_names.append(trans_name)
            pre_transform_col_names_values.append(x)
        except:
            pass
    file_mapping_dict = dict(zip(pre_transform_col_names, pre_transform_col_names_values))
    return file_mapping_dict


# Given a DF and specifying a field from that DF that contains JSON/dictionary data, this will append the JSON data to the existing DF
def dict_fields_to_df(original_df, dictionary_field_name): 
    dictionary_data = pd.DataFrame(list(original_df[dictionary_field_name]))
    original_df_field_names = list(original_df.columns.values)
    dictionary_field_names = list(dictionary_data.columns.values)
    for col in dictionary_field_names:
        if col in original_df_field_names:
            pass
        else:
            original_df[col] = dictionary_data[col]
    return original_df


# changes encoding/decoding to uft8 - helpful when solving "UnicodeDecodeError: 'utf8' codec can't decode byte...". Should only be used for cols with strings, allegedly.
# source: http://stackoverflow.com/questions/18645401/python-pandas-to-excel-utf8-codec-cant-decode-byte
def changeencode(data, cols):  
    for col in cols:
        data[col] = data[col].str.decode('iso-8859-1').str.encode('utf-8')
    return data 


# User input to specify which database to query
active_db = raw_input('Which Database (prod, uat, guat, redshift or devload)?  ')
while True:
    if str(active_db) == 'prod':
        db = DB(profile='production')
        break
    elif str(active_db) == 'uat':
        db = DB(profile='uat')
        break
    elif str(active_db) == 'redshift':
        db = DB(profile='redshift')
        break
    elif str(active_db) == 'devload':
        db = DB(profile='devload')
        break
    elif str(active_db) == 'guat':
        db = DB(profile='guat')
        break
    else:
        print 'Invalid response - please try again...'
        active_db = raw_input('Which Database (prod, uat, redshift or devload)? ')

# User input to specify buyer name - needed to know which C2FO Admin buyer config to look at
buyer_configs = db.query("""
                        select *
                        from fileset_configuration
                        ;"""
                        )
buyer_names_list = sorted(buyer_configs['upload_directory'].tolist()) # Prints a list of all possible buyer names
print buyer_names_list
buyer_name_input = raw_input('\nBuyer Name: ').lower()
while True:
    if buyer_name_input in buyer_names_list:
        buyer_config = buyer_configs[buyer_configs['upload_directory'] == buyer_name_input]
        buyer_index_value = list(buyer_config.index.values)
        break
    else:
        print 'Invalid response - please try again...'
        buyer_name_input = raw_input('\nBuyer Name: ').lower()

# Load buyer's outbound files to pandas dataframes 
dir_files = [f for f in listdir(user_config['base_path']) if isfile(join(user_config['base_path'], f))]
dir_files_final = [s for s in dir_files if s[0] != '~'] # Makes sure we don't pull an excel litter file

alternative_organization_filename = str(buyer_config['organization_filename'][buyer_index_value[0]]) #pull alternative eslap file names from buyer config in C2FO Admin
alternative_invoice_filename = str(buyer_config['invoice_filename'][buyer_index_value[0]])
alternative_user_filename = str(buyer_config['user_filename'][buyer_index_value[0]])

for file_name in dir_files_final:
    if ('invoice' in file_name.lower()) or (alternative_invoice_filename in file_name):
        invoice_file_name = file_name
        invoice_file_path = user_config['base_path'] + '\\' + file_name
        invoice_file = pd.read_csv(invoice_file_path)
    elif ('organization' in file_name.lower()) or (alternative_organization_filename in file_name):
        org_file_name = file_name
        org_file_path = user_config['base_path'] + '\\' + file_name
        org_file = pd.read_csv(org_file_path)
    elif ('user' in file_name.lower()) or (alternative_user_filename in file_name):
        user_file_name = file_name
        user_file_path = user_config['base_path'] + '\\' + file_name
        user_file = pd.read_csv(user_file_path)
    else:
        print 'There is at least one unknown file in %s' % (user_config['base_path'])

# Renaming *invoice file fields if the buyer does not send eslap compliant field names... this is done by checking the buyer config via the C2FO database 
# ToDo: extend this to over and user file fields
invoice_file_fields = invoice_file.columns.values.tolist()
eslap_invoice_fields = ['company_id', 
                 'division_id',
                 'invoice_id',
                 'amount',
                 'currency',
                 'payment_due_date',
                 'transaction_type',
                 'transaction_date',
                 'voucher_id',
                 'payment_term',
                 'payment_method',
                 'adj_invoice_id',
                 'adjustment_reason_code',
                 'vat_amount']

invoice_col_names_transformed = buyer_outbound_mappings(buyer_configs, buyer_name_input, 'invoice_map', eslap_invoice_fields) #Will need updating onces the scope of this code is extended to org/user files
final_col_name_list = []
for col in invoice_file_fields:
    if col in invoice_col_names_transformed:
        final_col_name_list.append(invoice_col_names_transformed[col])
    else:
        final_col_name_list.append(col)
invoice_file.rename(columns=dict(zip(invoice_file_fields, final_col_name_list)), inplace=True)
invoice_file['original_index'] = invoice_file.index # Creates new column to keep track of each row's original index (used in the creation of binary fields)

# Trim column names
# https://github.com/BrandonTerrebonne/Outbound-file-screener/issues/6
invoice_file_cols = list(invoice_file.columns.values)
new_invoice_file_cols = [x.strip() for x in invoice_file_cols]
invoice_file.columns = new_invoice_file_cols

# Formatting invoice file columns
invoice_file['amount'] = invoice_file['amount'].astype(str)
invoice_file['adj_invoice_id'] = invoice_file['adj_invoice_id'].astype(str)
invoice_file['invoice_id'] = invoice_file['invoice_id'].astype(str)

# Create two new DFs for each transaction type (1 and 2)
trans_type1_df = invoice_file[invoice_file['transaction_type'] == 1]
trans_type1_df['amount'] = trans_type1_df['amount'].astype(float)
trans_type2_df = invoice_file[invoice_file['transaction_type'] == 2]
trans_type2_df['amount'] = trans_type2_df['amount'].astype(float)
invoice_file_rowcount = len(invoice_file.index)
trans_type1_df_rowcount = len(trans_type1_df.index)
trans_type2_df_rowcount = len(trans_type2_df.index)

# Matched and unmatched adjustments
trans_type2_df['adj_invoice_id'] = trans_type2_df['adj_invoice_id'].astype(str)
unmatched_adj_df = trans_type2_df[trans_type2_df['adj_invoice_id'] == 'nan']
unmatched_count = len(unmatched_adj_df.index)
matched_adj_df = trans_type2_df[trans_type2_df['adj_invoice_id'] != 'nan']
matched_count = len(matched_adj_df.index)

# Future dated invoices, unmatched adjustments and matched adjustments
todays_date = datetime.today().strftime('%Y-%m-%d') # Get today's date in YYYY-MM-DD format
trans_type1_df['payment_due_date'] = trans_type1_df['payment_due_date'].astype(datetime) # Future dated invoices
future_dated_type1 = trans_type1_df[trans_type1_df['payment_due_date'] > todays_date]
future_dated_type1_count = len(future_dated_type1.index)
matched_adj_df['payment_due_date'] = matched_adj_df['payment_due_date'].astype(datetime) # Future dated matched adjustments
future_dated_matched_df = matched_adj_df[matched_adj_df['payment_due_date'] > todays_date]
future_dated_matched_count = len(future_dated_matched_df.index)
future_dated_matched_df_original_index_list = list(future_dated_matched_df['original_index'])
invoice_file['future_dated_matched_adj'] = np.where(invoice_file['original_index'].isin(future_dated_matched_df_original_index_list), 1, 0) 
unmatched_adj_df['payment_due_date'] = unmatched_adj_df['payment_due_date'].astype(datetime) # Future dated unmatched adjustments
future_dated_unmatched_df = unmatched_adj_df[unmatched_adj_df['payment_due_date'] > todays_date]
future_dated_unmatched_count = len(future_dated_unmatched_df.index) 
future_dated_unmatched_df_original_index_list = list(future_dated_unmatched_df['original_index']) 
invoice_file['future_dated_unmatched_adj'] = np.where(invoice_file['original_index'].isin(future_dated_unmatched_df_original_index_list), 1, 0) 

# Matched adjs where their corresponding invoices are missing from the invoice file
inv_id_list = invoice_file['invoice_id'].tolist()
adj_inv_id_list = matched_adj_df['adj_invoice_id'].tolist()
valid_matched_count = len([i for i in adj_inv_id_list if i in inv_id_list])
invalid_matched = [i for i in adj_inv_id_list if i not in inv_id_list]
invalid_matched_df = invoice_file[invoice_file['adj_invoice_id'].isin(invalid_matched)]
invalid_matched_df_original_index_list = list(invalid_matched_df['original_index']) 
invoice_file['matched_adj_missing_invoice'] = np.where(invoice_file['original_index'].isin(invalid_matched_df_original_index_list), 1, 0)

# Number of records with 'amount' equal to zero
invoice_file['amount'] = invoice_file['amount'].astype(float)
zero_amounts = invoice_file[invoice_file['amount'] == 0]
zero_amounts_count = len(zero_amounts.index)
zero_amounts_original_index_list = list(zero_amounts['original_index']) 
invoice_file['amount_is_zero'] = np.where(invoice_file['original_index'].isin(zero_amounts_original_index_list), 1, 0)

# Number of records incorrectly applying transaction_type logic
negative_trans_type1s = trans_type1_df[trans_type1_df['amount'] < 0] # Negative transaction_type = 1
negative_trans_type1s_original_index_list = list(negative_trans_type1s['original_index']) 
invoice_file['negative_type_1'] = np.where(invoice_file['original_index'].isin(negative_trans_type1s_original_index_list), 1, 0) 
positive_trans_type2s = trans_type2_df[trans_type2_df['amount'] > 0] # Positive transaction_type = 1
positive_trans_type2s_original_index_list = list(positive_trans_type2s['original_index']) 
invoice_file['positive_type_2'] = np.where(invoice_file['original_index'].isin(positive_trans_type2s_original_index_list), 1, 0) 

# Invoices with adjusted invoice IDs (aka matched invoices or invoice_with_adj_inv_id)
matched_inv_df = trans_type1_df[trans_type1_df['adj_invoice_id'] != 'nan']
matched_inv_count = len(matched_inv_df.index)
matched_inv_df_original_index_list = list(matched_inv_df['original_index']) 
invoice_file['invoice_with_adj_inv_id'] = np.where(invoice_file['original_index'].isin(matched_inv_df_original_index_list), 1, 0) 
    
# List all invoice file fields that are populated < 1 time
inv_file_fields = list(invoice_file.columns.values) # Get list of field names
empty_fields = []
inv_file_len = len(invoice_file.index) # Get row count of invoice file
for x in inv_file_fields: # Returns list of invoice file files that were not populated
    invoice_file[x] = invoice_file[x].astype(str)
    new_df = invoice_file[invoice_file[x] == 'nan']
    if len(new_df.index) == inv_file_len:
        empty_fields.append(x)

# Creating .xlsx doc to output the summary
summary_file_path = user_config['base_path'] + '\\' + 'OUTBOUND_FILE_SUMMARY_' + (time.strftime('%Y%m%d')) + '.xlsx'
workbook = xlsxwriter.Workbook(summary_file_path)
workbook.close()
wb = load_workbook(summary_file_path)
ws1 = wb.create_sheet('Summary', 0) # insert at first position
for i in range(1,101):
    for j in range(1,101):
        ws1.cell(row=i, column=j)
workbook = xlsxwriter.Workbook(summary_file_path)
worksheet = workbook.add_worksheet()
workbook.close()

# Writing Invoice File summary to Excel doc
ws1['A1'] = '--- Invoice File Summary ---'
ws1['A2'] = 'File name: %s' % (invoice_file_name)
ws1['A3'] = 'Number of records in the file: %s' % (invoice_file_rowcount)
ws1['A4'] = 'Number of invoices: %s' % (trans_type1_df_rowcount)
ws1['A5'] = 'Number of adjustments: %s' % (trans_type2_df_rowcount)
ws1['A6'] = 'Number of unmatched adjustments: %s' % (unmatched_count)
ws1['A7'] = 'Number of matched adjusments: %s' % (matched_count)
ws1['A9'] = '--- Age Summary for %s ---' % (todays_date)
ws1['A10'] = 'Number of future dated invoices: %s' % (future_dated_type1_count)
ws1['A11'] = 'Number of future dated unmatched adjustments: %s' % (future_dated_unmatched_count)
ws1['A12'] = 'Number of future dated matched adjustments: %s' % (future_dated_matched_count)
ws1['A14'] = '--- Issue Summary ---'
if len(invalid_matched_df.index) > 0:
    ws1['A15'] = 'WARNING - Number of matched adjustments missing their corresponding invoices: %s' % (len(invalid_matched_df.index))
else:
    ws1['A15'] = 'Passed - Number of matched adjustments missing their corresponding invoices: %s' % (len(invalid_matched_df.index))
if zero_amounts_count > 0:
    ws1['A16'] = 'WARNING - Number of records with \'amount\' equal to zero: %s' % (zero_amounts_count)
else:
    ws1['A16'] = 'Passed - Number of records with \'amount\' equal to 0.00: %s' % (zero_amounts_count)
if len(negative_trans_type1s.index) > 0:
    ws1['A17'] = 'WARNING - Number of NEGATIVE transaction_type 1s: %s' % (len(negative_trans_type1s.index))
else: 
    ws1['A17'] = 'Passed - Number of NEGATIVE transaction_type 1s: %s' % (len(negative_trans_type1s.index))
if len(positive_trans_type2s.index) > 0:
    ws1['A18'] = 'WARNING - Number of POSITIVE transaction_type 2s: %s' % (len(positive_trans_type2s.index))
else:
    ws1['A18'] = 'Passed - Number of POSITIVE transaction_type 2s: %s' % (len(positive_trans_type2s.index))
if matched_inv_count > 0:
    ws1['A19'] = "Warning - Number of transaction_type 1s with adj_invoice_ids: %s" % (matched_inv_count)
else:
    ws1['A19'] = "Passed - No invoices found with adj_invoice_id field populated"
if not empty_fields:
    ws1['A20'] = 'All fields are populated at least once.'
if empty_fields:
    ws1['A20'] = 'Warning - Fields not populated: %s' % (empty_fields)

# Writing Org File summary to Excel doc
ws1['A22'] = '--- Organization File Summary ---'
try:
    if org_file.index[0] != 0: # When data unintentionally shifts due to improper CSV format, the index will SOMETIMES default to the data that was intended for the first column. 
        ws1['A23'] = 'WARNING - %s may NOT be a valid CSV file!!!' % (org_file_name) # In which case, the assumption is that the first index value will not be 0
    else:
        ws1['A23'] = 'Passed - %s looks like a valid CSV file' % (org_file_name)
except (NameError, IndexError):
    ws1['A23'] = 'No organization file found or file has no data'

# Writing User File summary to Excel doc
ws1['A30'] = '--- User File Summary ---'
try:
    if user_file.index[0] != 0:
        ws1['A31'] = 'WARNING - %s may NOT be a valid CSV file!!!' % (user_file_name)
    else:
        ws1['A31'] = 'Passed - %s looks like a valid CSV file' % (user_file_name)
except (NameError, IndexError):
    ws1['A31'] = 'No user file found or file has no data'
wb.save(summary_file_path)

# DFs to excel worksheets... http://stackoverflow.com/questions/20219254/how-to-write-to-an-existing-excel-file-without-overwriting-data-using-pandas
book = load_workbook(summary_file_path)
writer = pd.ExcelWriter(summary_file_path, engine='openpyxl')
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
inv_file_tab_name = str(invoice_file_name)[:30]
invoice_file.replace(to_replace= 'nan', value= '', inplace= True)

# Fixes pd.to_excel issues with encoding/decoding - related to issue with anthem test file on 20170301
try:
    invoice_file.to_excel(writer, sheet_name= inv_file_tab_name, index=False)
except UnicodeDecodeError:
    invoice_file_cols = list(invoice_file.columns.values)
    invoice_file = changeencode(invoice_file, invoice_file_cols)
    invoice_file.to_excel(writer, sheet_name= inv_file_tab_name, index=False)

# Eligible AP @ supplier level
buyer_division_id = int(buyer_configs[buyer_configs['upload_directory'] == buyer_name_input]['maker_id'])
takers_markets_query = """
select mto.company_id, t.taker_maker_id as division_id, t.taker_division_name as company_name, t.eligible_invoice_amount as eligible_amount, 
t.eligible_invoice_count as eligible_count, t.matched_adjustment_count as matched_adj_count, t.unmatched_adjustment_count as unmatched_adj_count, 
t.currency, t.eligible_dpe_weighted_avg, t.min_eligible_dpe, t.max_eligible_dpe, t.is_participating, t.taker_id,  t.market_id, t.market_type
from takers_markets t
join makers_takers d
on t.taker_id = d.taker_id
full outer join makers_takers_organizations mto
on d.taker_organization_id = mto.taker_id
where t.maker_id = {} and eligible_invoice_count > 0
order by t.eligible_invoice_amount desc
;""".format(buyer_division_id)
takers_markets = db.query(takers_markets_query)
# If the buyer doesn't send division files, company_id and division_id 
# will be equal and we will drop the division_id series from the DF
if takers_markets['company_id'].equals(takers_markets['division_id']):
    takers_markets = takers_markets.drop('division_id', axis=1)
takers_markets.to_excel(writer, sheet_name= 'DB - Eligible AP', index= False)

#Invoice table - ToDo: ADD PAY_DATE!!!
invoices_query = """
select company_id, voucher_id as invoice_id, amount, earn, discounted_amount, adjustment_amount, vat_amount, due_date, cleared_date,
CAST(created as DATE), CAST(updated as DATE), currency, adj_invoice_id, transaction_type, is_cleared, is_awarded, is_paid, is_past_due, 
is_in_cash_pool, is_pending_clear, is_reserved, is_ineligible_ep, is_ineligible_ar, taker_excluded, maker_excluded, 
maker_excluded_for_today, taker_excluded_for_today, market_id, eslap
from invoice
where maker_id = {}
order by cleared_date desc
;""".format(buyer_division_id)
invoice_table_data = db.query(invoices_query)
dict_fields_to_df(invoice_table_data,'eslap')
invoice_table_data.drop('eslap', axis=1, inplace=True)
invoice_table_data.to_excel(writer, sheet_name= 'DB - Invoice Table', index= False)

# Supplier reserves - This will return an empty DF is no reserve data has been loaded
reserves_query = """
select mto.company_id, mt.division_id as division_id, mtr.taker_id as DB_taker_id, mtr.reserve_percentage, mtr.reserve_amount, mtr.invoice_priority
, mtr.run_before_adjustments, mtr.allow_eslap_updates, CAST(mtr.created as DATE), CAST(mtr.updated as DATE), mtr.reserve_reason
from market m
join market_taker_reserve mtr
on m.id = mtr.market_id
join makers_takers mt
on mt.taker_id = mtr.taker_id
full outer join makers_takers_organizations mto
on mt.taker_organization_id = mto.taker_id
where m.maker_id = {}
;""".format(buyer_division_id)
supplier_reserves = db.query(reserves_query)
# If the buyer doesn't send division files, company_id and division_id 
# will be equal and we will drop the division_id series from the DF
if supplier_reserves['company_id'].equals(supplier_reserves['division_id']):
    supplier_reserves = supplier_reserves.drop('division_id', axis=1)
supplier_reserves = supplier_reserves.drop_duplicates()
supplier_reserves.to_excel(writer, sheet_name= 'DB - Supplier Reserves', index= False)

 # Removes the random blank sheet that somehow always gets created (?!?)
sh= book.get_sheet_by_name('Sheet1')
book.remove_sheet(sh)
writer.save()

# Print summary (in case the invoice file is too big to open in excel)
print "\n---Invoice File Summary---\nFile name: %s" % (invoice_file_name)
print "Number of records in the file: %s" % (invoice_file_rowcount)
print "Number of invoices: %s" % (trans_type1_df_rowcount)
print "Number of adjustments: %s" % (trans_type2_df_rowcount)
print "Number of unmatched adjustments: %s" % (unmatched_count)
print "Number of matched adjusments: %s\n" % (matched_count)
print "---Invoice File Age Summary for %s---\nNumber of future dated invoices: %s" % (todays_date, future_dated_type1_count)
print "Number of future dated matched adjustments: %s" % (future_dated_matched_count)
print "Number of future dated unmatched adjustments:%s\n\n---Invoice File Issue Summary---" % (future_dated_unmatched_count)
if len(invalid_matched_df.index) > 0:
    print "Warning - Number of matched adjustments that do not have their corresponding invoices present: %s" % (len(invalid_matched_df.index))
else:
    print "Passed - All matched adjustments have their corresponding invoices present"
if zero_amounts_count > 0:
    print "Warning - Number of records with 'amount' equal to zero: %s" % (zero_amounts_count)
else:
    print "Passed - No records with amount = 0.00"
if len(negative_trans_type1s.index) > 0:
    print "Warning - Number of NEGATIVE transaction_type 1s: %s" % (len(negative_trans_type1s.index))
else:
    print "Passed - All transaction_type 1s are positive"
if len(positive_trans_type2s.index) > 0:
    print "Warning - Number of POSITIVE transaction_type 2s: %s" % (len(positive_trans_type2s.index))
else:
    print "Passed - All transaction_type 2s are negative"
if matched_inv_count > 0:
    print "Warning - Number of transaction_type 1s with adj_invoice_ids: %s" % (matched_inv_count)
else:
    print "Passed - No invoices found with adj_invoice_id field populated"
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Print currency pivot table --> ToDo: Figure out how to add this to the excel doc
invoice_file['amount'] = invoice_file['amount'].astype(float)
try:
    currency_pivot = invoice_file.pivot_table(index=['currency'], values=['amount'], aggfunc = np.sum)
    print "\n---Invoice File Currency Breakdown---\n%s" % (currency_pivot)
except KeyError:
    print "amount or currency field left blank for at least one record - cannot generate currency pivot"

print '\nDone!'

# Excel for costcouk =CONCAT(LEFT(E2,4),"-",MID(E2,5,2),"-",RIGHT(E2,2))